In [ ]:
from DistMLIP.implementations.matgl import TensorNet_Dist, Potential_Dist, MolecularDynamics, Relaxer
import matgl
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

In [ ]:
# Initialize a CHGNet_Dist model 
# because CHGNet_Dist inherits from matgl's CHGNet class: model finetuning, training, etc. all work the exact same as in MatGL
model = TensorNet_Dist()

# Load a previous CHGNet model from matgl
model = matgl.load_model("TensorNet-MatPES-PBE-v2025.1-PES").model

# Make a distributed version of the model
dist_model = TensorNet_Dist.from_existing(model)

# Enable distributed mode for 2 GPUs
dist_model.enable_distributed_mode([0, 1])

In [ ]:
# Insert your atoms
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5.5), ["Li", "Mn"], [[0, 0, 0], [0.5, 0.5, 0.5]])
struct.perturb(0.5)
struct.make_supercell((20, 20, 20))
print(f"There are {len(struct)} atoms.")
atoms = AseAtomsAdaptor().get_atoms(struct)

In [ ]:
# Create Potential_Dist object, use 128 threads when creating graph structures
potential = Potential_Dist(model=dist_model, num_threads=128)

In [ ]:
# Perform static point calculation
output = potential(atoms)

In [ ]:
# Run structure relaxation
relaxer = Relaxer(
    potential=potential,
    optimizer="FIRE",
    relax_cell=True
)

results = relaxer.relax(atoms, verbose=True, steps=200)

In [ ]:
# Run molecular dynamics
driver = MolecularDynamics(
    atoms,
    potential=potential,
    timestep=0.5,
    temperature=300,
    loginterval=200,
    logfile="logfile.log",
    trajectory="traj.trj"
)

driver.run(20)